In [ ]:
from ipyparallel import Client
import os
c = Client(profile='mpi')
v = c[:]
c.ids

# Libraries

In [ ]:
%%px
%matplotlib inline

import os
import time
import shutil
import numpy as np
import mpi4py.MPI as MPI

from matplotlib import cm
import matplotlib.pyplot as plt

import triangle

import pandas

from pyBadlands import xmlParser
from pyBadlands import FVmethod
from pyBadlands import diffLinear
from pyBadlands import raster2TIN 
from pyBadlands import flowNetwork
from pyBadlands import elevationTIN
from pyBadlands import partitionTIN
from pyBadlands import visualiseTIN
from pyBadlands import visualiseFlow
from pyBadlands import forceSim

import warnings
warnings.filterwarnings("ignore")

# Initialise MPI communications
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialisation Phase

In [ ]:
%%px
Init_time = time.clock()

In [ ]:
%%px
inputfile = '/home/ubuntu/mpiPyLEM/input.xml'
input = xmlParser.xmlParser(inputfile)

## 1. Get DEM regular grid and create Badlands TIN.

In [ ]:
%%px
walltime = time.clock()
tNow = input.tStart
recGrid = raster2TIN.raster2TIN(input.demfile,input.outDir,rank)

force = forceSim.forceSim(input.seafile,input.seapos,input.rainMap,input.rainTime,input.rainVal,
    input.tectFile,input.tectTime,recGrid.regX,recGrid.regY,input.tDisplay)

tinData = np.column_stack((recGrid.tinMesh['vertices'][:,0],
    recGrid.tinMesh['vertices'][:,1]))

if rank == 0:
    print " - read dataset and perform delaunay triangulation ", time.clock() - walltime

## 2. Partition the TIN

In [ ]:
%%px
walltime = time.clock()

# Set parameters of the finite volume mesh
FVmesh = FVmethod.FVmethod(recGrid.tinMesh['vertices'],
                  recGrid.tinMesh['triangles'],
                  recGrid.tinMesh['edges'])

# Perform partitioning by equivalent domain splitting
partitionIDs, RowProc, ColProc = partitionTIN.simple(tinData[:,0], tinData[:,1])
FVmesh.partIDs = partitionIDs

# Get each partition global node ID
inGIDs = np.where(partitionIDs == rank)[0]

if rank == 0:
    print " - partition TIN amongst processors ", time.clock() - walltime

## 3. Build Finite Volume discretisation

In [ ]:
%%px

# Define overlapping partitions
allIDs, localTIN = partitionTIN.overlap(tinData[:,0], \
    tinData[:,1], RowProc, ColProc, 2*recGrid.resEdges)

# Set parameters of the finite volume mesh
tMesh = FVmethod.FVmethod(localTIN['vertices'],
                  localTIN['triangles'],
                  localTIN['edges'])

walltime = time.clock()

# Define Finite Volume parameters
totPts = len(tinData[:,0])
FVmesh.neighbours = np.zeros((totPts,20), dtype=np.int) 
FVmesh.neighbours.fill(-2)
FVmesh.edge_lenght = np.zeros((totPts,20), dtype=np.float) 
FVmesh.vor_edges = np.zeros((totPts,20), dtype=np.float) 
FVmesh.control_volumes = np.zeros(totPts, dtype=np.float) 

# Compute Finite Volume parameters
tGIDs, tNgbh, tEdgs, tVors, tVols = tMesh.construct_FV(inGIDs, allIDs, totPts)

FVmesh.neighbours[tGIDs,:tMesh.maxNgbh] = tNgbh
FVmesh.edge_lenght[tGIDs,:tMesh.maxNgbh] = tEdgs
FVmesh.vor_edges[tGIDs,:tMesh.maxNgbh] = tVors
FVmesh.control_volumes[tGIDs] = tVols

if rank == 0:
    print " - FV mesh ", time.clock() - walltime

## 4. Interpolate elevation

In [ ]:
%%px
walltime = time.clock()
inIDs = np.where(FVmesh.partIDs[recGrid.boundsPt:] == rank)[0]
inIDs += recGrid.boundsPt
 
local_elev = np.zeros(totPts)
local_elev.fill(-1.e6)
local_elev[inIDs] = elevationTIN.getElevation(recGrid.regX, recGrid.regY, \
    recGrid.regZ, FVmesh.node_coords[inIDs,:2])
comm.Allreduce(MPI.IN_PLACE, local_elev, op=MPI.MAX)

xyMin = [recGrid.regX.min(),recGrid.regY.min()]
xyMax = [recGrid.regX.max(),recGrid.regY.max()]

elevation = elevationTIN.update_border_elevation(local_elev, \
    FVmesh.neighbours, FVmesh.edge_lenght, recGrid.boundsPt, btype=input.btype)

# Define variables
cumdiff = np.zeros(totPts)
hillslope = diffLinear()
hillslope.CDaerial = input.CDa
hillslope.CDmarine = input.CDm 
flow = flowNetwork()
flow.erodibility = input.SPLero
flow.m = input.SPLm
flow.n = input.SPLn
flow.mindt = input.minDT
    
if rank == 0:
    print " - interpolate elevation on grid ", time.clock() - walltime

In [ ]:
%%px
if rank == 0:
    print " - Initialisation phase ", time.clock() - Init_time

In [ ]:
%%px
step = 0
tNow = input.tStart
force.next_rain = input.tStart
force.next_disp = input.tStart
force.next_display = input.tStart
exitTime = tEnd

# Main loop

In [ ]:
%%px
# Load Rain Map
if force.next_rain <= tNow and force.next_rain < input.tEnd:
    rain = np.zeros(totPts, dtype=float)
    rain[inIDs] = force.load_Rain_map(tNow,FVmesh.node_coords[inIDs,:2])
    comm.Allreduce(MPI.IN_PLACE, rain, op=MPI.MAX)

# Load Tectonic Grid
if force.next_disp <= tNow and force.next_disp < input.tEnd:
    ldisp = np.zeros(totPts, dtype=float)
    ldisp.fill(-1.e6)
    ldisp[inIDs] = force.load_Tecto_map(tNow,FVmesh.node_coords[inIDs,:2])
    comm.Allreduce(MPI.IN_PLACE, ldisp, op=MPI.MAX)
    disp = force.disp_border(ldisp,FVmesh.neighbours,FVmesh.edge_lenght,recGrid.boundsPt)
    
#while ( tNow < force.next_disp and tNow < force.next_rain ):
#elif tNow == tEnd:
# print display
#else:
# update rain and disp maps and go back to while loop...

# Flow computation

In [ ]:
%%px
Flow_time = time.clock()

## 1. Perform pit filling

In [ ]:
%%px
walltime = time.clock()

# Update sea-level
force.getSea(tNow)
fillH = elevationTIN.pit_filling_PD(elevation, FVmesh.neighbours, \
                recGrid.boundsPt, force.sealevel-input.sealimit, input.fillmax)
if rank == 0:
    print " - depression-less algorithm PD with stack", time.clock() - walltime

## 2. Compute stream network

In [ ]:
%%px
walltime = time.clock()
ngbhs = FVmesh.neighbours[allIDs,:]
edges = FVmesh.vor_edges[allIDs,:]
distances = FVmesh.edge_lenght[allIDs,:]
flow.SFD_receivers(fillH, elevation, ngbhs, edges, distances, allIDs, 
                   force.sealevel-input.sealimit)
if rank == 0:
    print " - compute receivers parallel ", time.clock() - walltime

# Distribute evenly local minimas to processors
walltime = time.clock()
flow.localbase = np.array_split(flow.base, size)[rank]
flow.ordered_node_array()
if rank == 0:
    print " - compute stack order locally ", time.clock() - walltime
    
walltime = time.clock()
stackNbs = comm.allgather(len(flow.localstack))
globalstack = np.zeros(sum(stackNbs),dtype=flow.localstack.dtype)
comm.Allgatherv(sendbuf=[flow.localstack, MPI.INT], 
             recvbuf=[globalstack, (stackNbs, None), MPI.INT]) 
flow.stack = globalstack
if rank == 0:
    print " - send stack order globally ", time.clock() - walltime

## 3. Compute discharge

In [ ]:
%%px
walltime = time.clock()
flow.compute_flow(FVmesh.control_volumes, rain, True)
if rank == 0:
    print " - compute discharge ", time.clock() - walltime

In [ ]:
%%px
# Here we output the dataset if force.next_disp == tNow
if force.next_display <= tNow and force.next_display < input.tEnd:
    # get plotting function
    #step += 1
    force.next_display += force.time_display

exitTime = min(force.next_display,force.next_disp)
exitTime = min(exitTime,force.next_rain)

## 4. Compute CFL condition

In [ ]:
%%px
walltime = time.clock()  
hillslope.dt_pstability(FVmesh.edge_lenght[inGIDs,:tMesh.maxNgbh])
#
flow.dt_fstability(FVmesh.node_coords[:,:2],fillH,inGIDs)
#
CFLtime = min(flow.CFL,hillslope.CFL)
CFLtime = max(input.minDT,CFLtime)
if rank == 0:
    print " - Get CFL time step ", time.clock() - walltime

## 5. Compute sediment fluxes

In [ ]:
%%px
# Initial cumulative elevation change
walltime = time.clock()

diff_flux = hillslope.sedflux(flow.diff_flux,force.sealevel,elevation,
                              FVmesh.control_volumes)
if rank == 0:
    print " - Get hillslope fluxes ", time.clock() - walltime

walltime = time.clock()
tstep,sedrate = flow.compute_sedflux(FVmesh.control_volumes,elevation,fillH,
                FVmesh.node_coords[:,:2],xyMin,xyMax,diff_flux,
                CFLtime,force.sealevel,True)
if rank == 0:
    print " - Get stream fluxes ", time.clock() - walltime

# Update surface
timestep = min(tstep,exitTime-tNow)
elevation = elevation + sedrate * timestep
cumdiff = cumdiff + sedrate * timestep

In [ ]:
%%px
if rank == 0:
    print " - Flow computation ", time.clock() - Flow_time

# Create Hdf5 outputs

we first define local vertices, triangles and polylines used to create the visualisation outputs.

In [ ]:
%%px
out_time = time.clock()
visXlim = np.zeros(2)
visYlim = np.zeros(2)
visXlim[0] = recGrid.rectX.min()
visXlim[1] = recGrid.rectX.max()
visYlim[0] = recGrid.rectY.min()
visYlim[1] = recGrid.rectY.max()

# Done when TIN has been built/rebuilt
outPts, outCells = visualiseTIN.output_cellsIDs(allIDs,inIDs,visXlim,visYlim,
                                        FVmesh.node_coords[:,:2],tMesh.cells)
tcells = np.zeros(size)
tcells[rank] = len(outCells)
comm.Allreduce(MPI.IN_PLACE,tcells,op=MPI.MAX)
tnodes = np.zeros(size)
tnodes[rank] = len(allIDs)
comm.Allreduce(MPI.IN_PLACE,tnodes,op=MPI.MAX)

# Done for every visualisation step
flowIDs, polylines = visualiseFlow.output_Polylines(outPts,flow.receivers[outPts],
                visXlim,visYlim,FVmesh.node_coords[:,:2])
fnodes = np.zeros(size)
fnodes[rank] = len(flowIDs)
comm.Allreduce(MPI.IN_PLACE,fnodes,op=MPI.MAX)
fline = np.zeros(size)
fline[rank] = len(polylines[:,0])
comm.Allreduce(MPI.IN_PLACE,fline,op=MPI.MAX)

we then write to each partition simulation outputs (TIN & flow network) using HdF5.

In [ ]:
%%px
# Compute flow parameters
flow.compute_parameters(FVmesh.node_coords[:,:2])

# Write HDF5 files
visualiseTIN.write_hdf5(input.outDir,input.th5file,step,tMesh.node_coords[:,:2],
                        elevation[allIDs],flow.discharge[allIDs],cumdiff[allIDs],
                        outCells,rank)
visualiseFlow.write_hdf5(input.outDir,input.fh5file,step,FVmesh.node_coords[flowIDs,:2],
                         elevation[flowIDs],flow.discharge[flowIDs],flow.chi[flowIDs],
                         flow.basinID[flowIDs],polylines,rank)

# Combine HDF5 files and write time series
if rank==0:
    visualiseTIN.write_xmf(input.outDir,input.txmffile,input.txdmffile,
                           step,input.tStart,tcells,tnodes,input.th5file,size)
    visualiseFlow.write_xmf(input.outDir,input.fxmffile,input.fxdmffile,
                            step,input.tStart,fline,fnodes,input.fh5file,size)
    print " - Writing outputs ", time.clock() - out_time